<a href="https://colab.research.google.com/github/DarianSawali/News-Based-RAG/blob/main/news_rag_gpt2_summary%20update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install feedparser newspaper3k sentence-transformers faiss-cpu


  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 31.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 2.3 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13540 sha256=4bbdf4a207d5d0b5d6ce16c1e43020d45a3535799c79180b981709e3c1d3041f
  Stored in directory: /root/.cache/pip/wheels/a5/91/9f/00d66475960891a64867914273fcaf78df6cb04d905b104a2a
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3341 sha256=0bceeaf0a0cb5d3e5ca0857f1c5eb81c5f689fda91f1e8b158ae80c6e6b0ac57
  Stored in 

In [ ]:
!pip install lxml_html_clean


In [ ]:
!pip install requests


In [ ]:
import requests
import feedparser

def safe_feedparse(url, timeout=5):
    """Fetch RSS with a timeout so it can’t stall the notebook."""
    try:
        resp = requests.get(url, timeout=timeout, headers={"User-Agent": "Mozilla/5.0"})
        resp.raise_for_status()
        return feedparser.parse(resp.text)
    except Exception as e:
        print("Failed to fetch:", url, "→", e)
        return feedparser.parse("")


In [ ]:
def fetch_articles_from_rss(source_name, feed_url, max_articles=20):
    feed = safe_feedparse(feed_url)
    print(source_name, "| entries in feed:", len(feed.entries))

    docs = []
    for entry in feed.entries[:max_articles]:
        title = entry.get("title", "").strip()
        summary = entry.get("summary", "").strip()
        url = entry.get("link", "")

        text = summary or title
        if not text:
            continue

        docs.append({
            "source": source_name,
            "title": title,
            "text": text,
            "url": url,
            "published": entry.get("published", "")
        })
    return docs


In [ ]:
NEWS_SOURCES = {
    "Global News BC": "https://globalnews.ca/bc/feed/",
    "CBC BC": "https://www.cbc.ca/webfeed/rss/rss-canada-britishcolumbia",
    "The Province": "https://theprovince.com/feed",
}

all_documents = []
for name, url in NEWS_SOURCES.items():
    docs = fetch_articles_from_rss(name, url, max_articles=20)
    print(" → Retrieved:", len(docs))
    all_documents.extend(docs)

print("Total articles:", len(all_documents))


Global News BC | entries in feed: 10
 → Retrieved: 10
CBC BC | entries in feed: 20
 → Retrieved: 20
The Province | entries in feed: 10
 → Retrieved: 10
Total articles: 40


In [ ]:
import feedparser

NEWS_SOURCES = {
    # "CBC BC": "https://www.cbc.ca/cmlink/rss-canada-britishcolumbia",
    "Global News BC": "https://globalnews.ca/bc/feed/",
    # "CTV Vancouver": "https://bc.ctvnews.ca/rss/ctv-news-vancouver-1.822295",
    "CBC BC": "https://www.cbc.ca/webfeed/rss/rss-canada-britishcolumbia",
    "The Province": "https://theprovince.com/feed",
    "City of Vancouver": "<city-hall-news-rss-url>",
}

def fetch_articles_from_rss(source_name, feed_url, max_articles=50):
    feed = feedparser.parse(feed_url)
    docs = []

    for entry in feed.entries[:max_articles]:
        title = entry.get("title", "").strip()
        summary = entry.get("summary", "").strip()
        url = entry.get("link", "")

        text = summary or title

        if not text:
            continue

        docs.append({
            "source": source_name,
            "title": title,
            "text": text,
            "url": url,
            "published": entry.get("published", "")
        })

    return docs


In [ ]:
import feedparser

feed = feedparser.parse("https://globalnews.ca/bc/feed/")
print(len(feed.entries))

10


In [ ]:
import numpy as np
import re
from sklearn.metrics.pairwise import cosine_similarity

def extractive_summary(text, n_sentences=2):
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())
    sentences = [s for s in sentences if s]
    if len(sentences) <= n_sentences:
        return " ".join(sentences)

    sent_embs = embed_model.encode(sentences, convert_to_numpy=True)
    doc_emb = embed_model.encode([text], convert_to_numpy=True)[0]

    sims = cosine_similarity(sent_embs, doc_emb.reshape(1, -1)).flatten()

    top_idx = np.argsort(sims)[-n_sentences:]
    top_idx = sorted(top_idx)
    selected = [sentences[i] for i in top_idx]

    return " ".join(selected)


Fetching Global News BC
Global News BC | entries in feed: 10
 → Retrieved: 10
Fetching CBC BC
CBC BC | entries in feed: 20
 → Retrieved: 20
Fetching The Province
The Province | entries in feed: 10
 → Retrieved: 10
Total articles: 40


In [ ]:
len(all_documents), all_documents[1]

(40,
 {'source': 'Global News BC',
  'title': 'Pressure rises to shut down Granville Street SROs with emergency calls up 822%, HVA says',
  'text': "Vancouver's hospitality industry continues to put pressure on the provincial government over the deteriorating situation on Granville Street.",
  'url': 'https://globalnews.ca/news/11547146/pressure-shut-granville-street-sro-emergency-calls-up/',
  'published': 'Fri, 28 Nov 2025 00:19:58 +0000'})

In [ ]:
!pip install sentence-transformers faiss-cpu

from sentence_transformers import SentenceTransformer
import numpy as np
import faiss


In [ ]:
embed_model = SentenceTransformer("all-MiniLM-L6-v2")

corpus_texts = [
    doc["title"] + "\n\n" + doc["text"]
    for doc in all_documents
]

# Create embeddings
corpus_embeddings = embed_model.encode(corpus_texts, convert_to_numpy=True)
corpus_embeddings = corpus_embeddings.astype("float32")

print("Number of docs:", len(corpus_texts))
print("Embedding shape:", corpus_embeddings.shape)


Number of docs: 40
Embedding shape: (40, 384)


In [ ]:
embedding_dim = corpus_embeddings.shape[1]

index = faiss.IndexFlatL2(embedding_dim)
index.add(corpus_embeddings)

print("FAISS index size:", index.ntotal)


FAISS index size: 40


In [ ]:
def retrieve_docs(query, k=3):

    q_emb = embed_model.encode([query], convert_to_numpy=True).astype("float32")

    distances, indices = index.search(q_emb, k)

    results = []
    for dist, idx in zip(distances[0], indices[0]):
        doc = all_documents[idx]
        doc = {**doc, "distance": float(dist)}
        results.append(doc)
    return results


In [ ]:
test_query = "traffic delays in Metro Vancouver"
retrieved = retrieve_docs(test_query, k=3)

for r in retrieved:
    print(r["source"], "-", r["title"])
    print("distance:", r["distance"])
    print("url:", r["url"])
    print()


CBC BC - Vancouver police ask residents to be vigilant, report extortion as cases rise elsewhere
distance: 1.269562005996704
url: https://www.cbc.ca/news/canada/british-columbia/vancouver-police-department-extortion-forum-9.6994729?cmp=rss

Global News BC - Pressure rises to shut down Granville Street SROs with emergency calls up 822%, HVA says
distance: 1.3734512329101562
url: https://globalnews.ca/news/11547146/pressure-shut-granville-street-sro-emergency-calls-up/

Global News BC - B.C. communities see sudden spike in overdoses and an animal drug may be to blame
distance: 1.4454901218414307
url: https://globalnews.ca/news/11548094/bc-sudden-spike-overdoses-animal-drug-medetomidine/



In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
gpt2 = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")
tokenizer.pad_token = tokenizer.eos_token


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# def summarize_query_with_rag(query, k=3, max_new_tokens=80):

#     retrieved = retrieve_docs(query, k=k)

#     if len(retrieved) == 0:
#         return "No relevant news found for this query."

#     all_summaries = []

#     for doc in retrieved:
#         source = doc["source"]
#         title = doc["title"]
#         text = doc["text"]

#         prompt = f"""
# You are a STRICT summarization system.

# RULES:
# - Summarize ONLY THIS ONE article.
# - Do NOT add any new facts.
# - Do NOT mix information from any other article.
# - If something is not mentioned in the text, you MUST NOT mention it.
# - Write 1–2 sentences.
# - End the summary with "(Source: {source})".

# ARTICLE:
# [{source}] {title}
# {text}

# SUMMARY:
# """.strip()

#         inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)

#         output_ids = gpt2.generate(
#             **inputs,
#             max_new_tokens=min(max_new_tokens, 60),
#             do_sample=False,
#             repetition_penalty=1.2,
#             pad_token_id=tokenizer.eos_token_id,
#         )

#         generated_only = output_ids[0][inputs["input_ids"].shape[1]:]
#         summary = tokenizer.decode(generated_only, skip_special_tokens=True).strip()
#         all_summaries.append(summary)

#     return "\n\n".join(all_summaries)


In [ ]:
# import re

# def extract_first_sentences(text, n=2):
#     """Return the first n sentences from a block of text."""
#     sentences = re.split(r'(?<=[.!?])\s+', text.strip())
#     sentences = [s for s in sentences if s]
#     return " ".join(sentences[:n])


# def summarize_query_with_rag(query, k=3):
#     retrieved = retrieve_docs(query, k=k)

#     if len(retrieved) == 0:
#         return "No relevant news found for this query."

#     summaries = []

#     for doc in retrieved:
#         source = doc["source"]
#         title = doc["title"]
#         text = doc["text"]

#         short = extract_first_sentences(text, n=2)

#         summary = f"{short} (Source: {source} — {title})"
#         summaries.append(summary)

#     return "\n\n".join(summaries)

In [ ]:
from dateutil import parser as dateparser

def get_published_datetime(doc):
    pub = doc.get("published", "")
    try:
        return dateparser.parse(pub)
    except Exception:
        return None


In [ ]:
from dateutil import parser as dateparser
from datetime import timezone

def get_published_datetime(doc):
    pub = doc.get("published", "")
    if not pub:
        return None
    try:
        dt = dateparser.parse(pub)
        if dt is None:
            return None

        if dt.tzinfo is not None:
            dt = dt.astimezone(timezone.utc).replace(tzinfo=None)

        return dt
    except Exception:
        return None


In [ ]:
def summarize_latest_news(n=5):
    dated_docs = []
    for d in all_documents:
        dt = get_published_datetime(d)
        if dt is not None:
            dated_docs.append((dt, d))

    if not dated_docs:
        return "No recent articles available."

    dated_docs.sort(key=lambda x: x[0], reverse=True)

    top_docs = [d for _, d in dated_docs[:n]]

    summaries = []
    for doc in top_docs:
        source = doc["source"]
        title = doc["title"]
        text = doc["text"]
        url = doc.get("url", "")

        short = extractive_summary(text, n_sentences=2)
        if not short:
            continue

        line = f"{short} (Source: {source} — {title})"
        if url:
            line += f"\nLink: {url}"
        summaries.append(line)

    if not summaries:
        return "No recent articles available."

    return "\n\n".join(summaries)


In [ ]:
print(summarize_latest_news(n=5))


Vancouver fire said due to the exposure to "extreme levels of human suffering", firefighters are limited to 81 shifts at Firehall 2 to try and limit compassion fatigue. (Source: Global News BC — B.C. communities see sudden spike in overdoses and an animal drug may be to blame)
Link: https://globalnews.ca/news/11548094/bc-sudden-spike-overdoses-animal-drug-medetomidine/

Vancouver's hospitality industry continues to put pressure on the provincial government over the deteriorating situation on Granville Street. (Source: Global News BC — Pressure rises to shut down Granville Street SROs with emergency calls up 822%, HVA says)
Link: https://globalnews.ca/news/11547146/pressure-shut-granville-street-sro-emergency-calls-up/

David Kampf, Aatu Raty and Max Sasson are perfectly competent NHL centres but none are what the Canucks need: a second line centre. (Source: The Province — Canucks’ centre conundrum remains as Aatu Raty goes for a reset)
Link: https://theprovince.com/sports/hockey/nhl/va

In [ ]:
def search_articles(query, k=5):
    """
    Retrieve top-k articles for a query and print them as a list the user can pick from.
    Returns the list of retrieved docs so you can reuse it.
    """
    retrieved = retrieve_docs(query, k=k)
    if len(retrieved) == 0:
        print("No relevant news found for this query.")
        return []

    for idx, doc in enumerate(retrieved):
        print(f"[{idx}] {doc['title']} ({doc['source']})")
        if doc.get("published"):
            print("     Published:", doc["published"])
        if doc.get("url"):
            print("     Link:", doc["url"])
        print()
    return retrieved


In [ ]:
candidates = search_articles("pipeline northern BC", k=5)


[0] Two key North Coast B.C. leaders meet with Alberta premier ahead of federal pipeline MOU (CBC BC)
     Published: Wed, 26 Nov 2025 18:03:00 EST
     Link: https://www.cbc.ca/news/canada/british-columbia/kitimat-haisla-alberta-oil-pipeline-9.6993855?cmp=rss

[1] Proposed pipeline through B.C. can’t distract from province’s other projects, Eby says (CBC BC)
     Published: Thu, 27 Nov 2025 11:12:10 EST
     Link: https://www.cbc.ca/news/politics/livestory/alberta-ottawa-memorandum-of-understanding-energy-deal-pipeline-bc-9.6993431?cmp=rss

[2] B.C. premier calls pipeline MOU an ‘energy vampire’, First Nations call it a ‘pipe dream’ (Global News BC)
     Published: Thu, 27 Nov 2025 19:35:00 +0000
     Link: https://globalnews.ca/news/11546713/bc-reaction-david-eby-pipeline-first-nations/

[3] Review of B.C.'s emissions recommends renewed efforts as critics say LNG stands in way of progress (CBC BC)
     Published: Wed, 26 Nov 2025 18:08:25 EST
     Link: https://www.cbc.ca/news/canada

In [ ]:
def summarize_doc(doc, n_sentences=2):
    source = doc["source"]
    title = doc["title"]
    text = doc["text"]
    url = doc.get("url", "")

    short = extractive_summary(text, n_sentences=n_sentences)
    if not short:
        return "No content to summarize."

    line = f"{short} (Source: {source} — {title})"
    if url:
        line += f"\nLink: {url}"
    return line


In [ ]:
print(summarize_doc(candidates[0]))


<img alt="A distant shot of a freighter pulled by a tugboat." height="349" src="https://i.cbc.ca/ais/1.7501205,1764208858801/full/max/0/default.jpg?im=Crop%2Crect%3D%280%2C252%2C4120%2C2317%29%3BResize%3D%28620%29" title="The Maran Gas Roxana LNG carrier navigates its way through the Douglas Channel to Kitimat, B.C., on Wednesday, April 2, 2025." width="620" /><p>The mayor of Kitimat, B.C., and elected chief of the Haisla First Nation say they reminded Alberta Premier Danielle Smith of the longstanding opposition to an oil pipeline in the region.</p> (Source: CBC BC — Two key North Coast B.C. leaders meet with Alberta premier ahead of federal pipeline MOU)
Link: https://www.cbc.ca/news/canada/british-columbia/kitimat-haisla-alberta-oil-pipeline-9.6993855?cmp=rss


In [ ]:
def answer_news_query(query):
    q = query.lower().strip()

    if any(word in q for word in ["latest", "recent", "today", "this morning", "this evening"]):
        return summarize_latest_news(n=5)

    if q.startswith("summarize article") or q.startswith("summarize story"):
        stripped = (
            q.replace("summarize article", "")
             .replace("summarize story", "")
             .strip()
        )
        if not stripped:
            stripped = query
        single_summary = summarize_query_with_rag(stripped, k=1)
        return single_summary

    return summarize_query_with_rag(query, k=3)


In [ ]:
print(answer_news_query("What are the latest stories?"))

print(answer_news_query("summarize article about the B.C. pipeline MOU"))

print(answer_news_query("summarize story on coastal First Nations and the tanker ban"))

print(answer_news_query("translink delays in vancouver"))


Vancouver fire said due to the exposure to "extreme levels of human suffering", firefighters are limited to 81 shifts at Firehall 2 to try and limit compassion fatigue. (Source: Global News BC — B.C. communities see sudden spike in overdoses and an animal drug may be to blame)
Link: https://globalnews.ca/news/11548094/bc-sudden-spike-overdoses-animal-drug-medetomidine/

Vancouver's hospitality industry continues to put pressure on the provincial government over the deteriorating situation on Granville Street. (Source: Global News BC — Pressure rises to shut down Granville Street SROs with emergency calls up 822%, HVA says)
Link: https://globalnews.ca/news/11547146/pressure-shut-granville-street-sro-emergency-calls-up/

David Kampf, Aatu Raty and Max Sasson are perfectly competent NHL centres but none are what the Canucks need: a second line centre. (Source: The Province — Canucks’ centre conundrum remains as Aatu Raty goes for a reset)
Link: https://theprovince.com/sports/hockey/nhl/va

In [ ]:
def chat_loop():
    print("Vancouver News Assistant. Type 'quit' to exit.\n")
    while True:
        q = input("You: ")
        if q.lower().strip() in ["quit", "exit"]:
            print("Bye!")
            break
        answer = answer_news_query(q)
        print("\nAssistant:\n" + answer + "\n")
